In [3]:
rm( list=ls() )  #Borro todos los objetos
gc()   #Garbage Collection

require("data.table")
require("rpart")
require("parallel")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,616221,33.0,1321371,70.6,1321371,70.6
Vcells,1097347,8.4,8388608,64.0,1632151,12.5


Loading required package: data.table

Warning message:
"package 'data.table' was built under R version 4.2.3"
Loading required package: rpart

Warning message:
"package 'rpart' was built under R version 4.2.3"
Loading required package: parallel



In [4]:

ksemillas  <- c(431020, 391879, 267595, 149458, 373138, 642844, 448558, 777843, 490488, 39023, 776783, 337476, 410606, 463004, 459064, 555741, 615950, 441623, 599738, 193603 ) #reemplazar por las propias semillas


In [5]:
#particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30 


In [6]:

particionar  <- function( data,  division, agrupa="",  campo="fold", start=1, seed=NA )
{
  if( !is.na(seed) )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x )} ,   division,  seq( from=start, length.out=length(division) )  ) )  

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
          by= agrupa ]
}


In [7]:
ArbolEstimarGanancia  <- function( semilla, param_basicos )
{
  #particiono estratificadamente el dataset
  particionar( dataset, division=c(7,3), agrupa="clase_ternaria", seed= semilla )  #Cambiar por la primer semilla de cada uno !

  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .",     #quiero predecir clase_ternaria a partir del resto
                   data= dataset[ fold==1],  #fold==1  es training,  el 70% de los datos
                   xval= 0,
                   control= param_basicos )  #aqui van los parametros del arbol

  #aplico el modelo a los datos de testing
  prediccion  <- predict( modelo,   #el modelo que genere recien
                          dataset[ fold==2],  #fold==2  es testing, el 30% de los datos
                          type= "prob") #type= "prob"  es que devuelva la probabilidad

  #prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  #cada columna es el vector de probabilidades 


  #calculo la ganancia en testing  qu es fold==2
  ganancia_test  <- dataset[ fold==2, 
                             sum( ifelse( prediccion[, "BAJA+2"]  >  0.025,
                                         ifelse( clase_ternaria=="BAJA+2", 117000, -3000 ),
                                         0 ) )]

  #escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada  <-  ganancia_test / 0.3

  return( list( "testing"=       dataset[ fold==2, .N],
                "testing_pos"=   dataset[ fold==2 & clase_ternaria=="BAJA+2", .N],
                "envios"=        dataset[ fold==2 , sum( prediccion[ , "BAJA+2"] > 0.025)],
                "aciertos"=      dataset[ fold==2, sum( prediccion[ , "BAJA+2"] > 0.025 & clase_ternaria=="BAJA+2" )],
                "ganancia_test"= ganancia_test_normalizada ))
}

In [8]:

#Aqui se debe poner la carpeta de la computadora local
setwd("C:/Users/ccarl/Documents/labo2023v/src")   #Establezco el Working Directory
#cargo los datos


In [9]:

#cargo los datos
dataset  <- fread("./datasets/dataset_pequeno.csv")


In [10]:

#trabajo solo con los datos con clase, es decir 202107
dataset  <- dataset[ clase_ternaria!= "" ]



In [11]:

param_basicos  <- list( "cp"=          -1,  #complejidad minima
                        "minsplit"=   900,  #minima cantidad de registros en un nodo para hacer el split
                        "minbucket"=  440,  #minima cantidad de registros en una hoja
                        "maxdepth"=     5 ) #profundidad máxima del arbol



In [12]:
#Un solo llamado, con la semilla 17
ArbolEstimarGanancia( 17, param_basicos )   



$testing
[1] 49405

$testing_pos
[1] 380

$envios
[1] 3899

$aciertos
[1] 231

$ganancia_test
[1] 53410000

In [13]:

#la funcion mcmapply  llama a la funcion ArbolEstimarGanancia  tantas veces como valores tenga el vector  ksemillas
salidas  <- mcmapply( ArbolEstimarGanancia, 
                      ksemillas,   #paso el vector de semillas, que debe ser el primer parametro de la funcion ArbolEstimarGanancia
                      MoreArgs= list( param_basicos),  #aqui paso el segundo parametro
                      SIMPLIFY= FALSE,
                      mc.cores= 1 )  #se puede subir a 5 si posee Linux o Mac OS

#muestro la lista de las salidas en testing para la particion realizada con cada semilla
salidas


[[1]]
[[1]]$testing
[1] 49405

[[1]]$testing_pos
[1] 380

[[1]]$envios
[1] 3466

[[1]]$aciertos
[1] 234

[[1]]$ganancia_test
[1] 58940000


[[2]]
[[2]]$testing
[1] 49401

[[2]]$testing_pos
[1] 379

[[2]]$envios
[1] 3795

[[2]]$aciertos
[1] 219

[[2]]$ganancia_test
[1] 49650000


[[3]]
[[3]]$testing
[1] 49406

[[3]]$testing_pos
[1] 381

[[3]]$envios
[1] 3786

[[3]]$aciertos
[1] 228

[[3]]$ganancia_test
[1] 53340000


[[4]]
[[4]]$testing
[1] 49404

[[4]]$testing_pos
[1] 379

[[4]]$envios
[1] 3933

[[4]]$aciertos
[1] 230

[[4]]$ganancia_test
[1] 52670000


[[5]]
[[5]]$testing
[1] 49407

[[5]]$testing_pos
[1] 381

[[5]]$envios
[1] 3879

[[5]]$aciertos
[1] 235

[[5]]$ganancia_test
[1] 55210000


[[6]]
[[6]]$testing
[1] 49405

[[6]]$testing_pos
[1] 380

[[6]]$envios
[1] 3862

[[6]]$aciertos
[1] 225

[[6]]$ganancia_test
[1] 51380000


[[7]]
[[7]]$testing
[1] 49406

[[7]]$testing_pos
[1] 381

[[7]]$envios
[1] 4063

[[7]]$aciertos
[1] 241

[[7]]$ganancia_test
[1] 55770000


[[8]]
[[8]]$testing
[1] 49405

[[8]]$testing_pos
[1] 380

[[8]]$envios
[1] 3517

[[8]]$aciertos
[1] 208

[[8]]$ganancia_test
[1] 48030000


[[9]]
[[9]]$testing
[1] 49404

[[9]]$testing_pos
[1] 380

[[9]]$envios
[1] 4475

[[9]]$aciertos
[1] 242

[[9]]$ganancia_test
[1] 52050000


[[10]]
[[10]]$testing
[1] 49405

[[10]]$testing_pos
[1] 380

[[10]]$envios
[1] 3877

[[10]]$aciertos
[1] 226

[[10]]$ganancia_test
[1] 51630000


[[11]]
[[11]]$testing
[1] 49404

[[11]]$testing_pos
[1] 380

[[11]]$envios
[1] 3949

[[11]]$aciertos
[1] 233

[[11]]$ganancia_test
[1] 53710000


[[12]]
[[12]]$testing
[1] 49406

[[12]]$testing_pos
[1] 381

[[12]]$envios
[1] 4305

[[12]]$aciertos
[1] 235

[[12]]$ganancia_test
[1] 50950000


[[13]]
[[13]]$testing
[1] 49406

[[13]]$testing_pos
[1] 381

[[13]]$envios
[1] 3803

[[13]]$aciertos
[1] 249

[[13]]$ganancia_test
[1] 61570000


[[14]]
[[14]]$testing
[1] 49403

[[14]]$testing_pos
[1] 380

[[14]]$envios
[1] 3641

[[14]]$aciertos
[1] 238

[[14]]$ganancia_test
[1] 58790000


[[15]]
[[15]]$testing
[1] 49405

[[15]]$testing_pos
[1] 381

[[15]]$envios
[1] 3754

[[15]]$aciertos
[1] 224

[[15]]$ganancia_test
[1] 52060000


[[16]]
[[16]]$testing
[1] 49405

[[16]]$testing_pos
[1] 380

[[16]]$envios
[1] 3794

[[16]]$aciertos
[1] 223

[[16]]$ganancia_test
[1] 51260000


[[17]]
[[17]]$testing
[1] 49403

[[17]]$testing_pos
[1] 379

[[17]]$envios
[1] 3793

[[17]]$aciertos
[1] 221

[[17]]$ganancia_test
[1] 50470000


[[18]]
[[18]]$testing
[1] 49406

[[18]]$testing_pos
[1] 380

[[18]]$envios
[1] 3535

[[18]]$aciertos
[1] 213

[[18]]$ganancia_test
[1] 49850000


[[19]]
[[19]]$testing
[1] 49405

[[19]]$testing_pos
[1] 381

[[19]]$envios
[1] 3875

[[19]]$aciertos
[1] 230

[[19]]$ganancia_test
[1] 53250000


[[20]]
[[20]]$testing
[1] 49405

[[20]]$testing_pos
[1] 381

[[20]]$envios
[1] 3001

[[20]]$aciertos
[1] 190

[[20]]$ganancia_test
[1] 45990000

In [14]:

#paso la lista a vector
tb_salida  <- rbindlist(salidas)

tb_salida


testing,testing_pos,envios,aciertos,ganancia_test
<int>,<int>,<int>,<int>,<dbl>
49405,380,3466,234,58940000
49401,379,3795,219,49650000
49406,381,3786,228,53340000
49404,379,3933,230,52670000
49407,381,3879,235,55210000
49405,380,3862,225,51380000
49406,381,4063,241,55770000
49405,380,3517,208,48030000
49404,380,4475,242,52050000


In [15]:

#finalmente calculo la media (promedio)  de las ganancias
tb_salida[  , mean( ganancia_test ) ]


[1] 52828500

In [16]:

#calculo todos los promedios
tb_salida[  , lapply( .SD, mean ) ]

#desvio estandar Distribucion Binomial   sqrt( n * p * (1-p) )


testing,testing_pos,envios,aciertos,ganancia_test
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
49404.8,380.25,3805.15,227.2,52828500
